### Step:1 Import data from binance


In [51]:
# ใช้สำหรับการ import data from binance
import ccxt
import pandas as pd

In [52]:
# สร้างเชื่อมต่อกับ Binance
exchange = ccxt.binance()
symbol = 'BTC/USD'  # คู่เหรียญ
timeframe = '1d'  # กรอบเวลา: 1 วัน

Import data from 200 days before to train the algorithms.

In [53]:
# ดึงข้อมูลย้อนหลัง 
data = exchange.fetch_ohlcv(symbol, timeframe, limit=300)  # ล่าสุด 300 แท่ง
df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']) #ปรับเพื่อแก้ว่าต้องการข้อมูลอะไรเพิ่ม
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
print(df.head())

   timestamp     open     high      low    close        volume
0 2024-07-23  67536.0  67752.8  65432.1  65926.8  29184.184218
1 2024-07-24  65926.8  67093.5  65052.0  65347.8  21394.867329
2 2024-07-25  65344.9  66146.3  63413.3  65764.3  31885.959601
3 2024-07-26  65764.4  68190.8  65687.3  67894.3  27183.398187
4 2024-07-27  67894.4  69412.4  66618.1  67864.2  39807.939686


In [54]:
print(df)

     timestamp      open      high      low     close        volume
0   2024-07-23   67536.0   67752.8  65432.1   65926.8  29184.184218
1   2024-07-24   65926.8   67093.5  65052.0   65347.8  21394.867329
2   2024-07-25   65344.9   66146.3  63413.3   65764.3  31885.959601
3   2024-07-26   65764.4   68190.8  65687.3   67894.3  27183.398187
4   2024-07-27   67894.4   69412.4  66618.1   67864.2  39807.939686
..         ...       ...       ...      ...       ...           ...
195 2025-02-03   97614.2  102511.0  91000.0  101382.0  74190.353191
196 2025-02-04  101382.0  101776.5  96058.3   97752.0  40724.480363
197 2025-02-05   97752.1   99177.5  96052.6   96569.6  22111.746163
198 2025-02-06   96560.2   99137.9  95660.6   96531.6  18820.033799
199 2025-02-07   96530.3   97846.9  96391.4   97397.4   5708.909622

[200 rows x 6 columns]


เพิ่ม Technical terms บางอย่าง เช่น

`macd` เป็นอินดิเคเตอร์ที่ใช้วัดความสัมพันธ์ระหว่างค่าเฉลี่ยเคลื่อนที่ (Moving Averages) สองตัว เป็นของ EMA 12 - EMA 26

`macd_signal` คือค่าเฉลี่ยเคลื่อนที่ของเส้น MACD

`macd_diff` value between MACD - MACDsignal

`RSI`วัด value แรงซื้อหรือแรงขายในตลาด

In [55]:
from ta.momentum import RSIIndicator
from ta.trend import MACD

# คำนวณ RSI
rsi = RSIIndicator(close=df['close'], window=14)
df['rsi'] = rsi.rsi()

# คำนวณ MACD
macd = MACD(close=df['close'])
df['macd'] = macd.macd()  # ค่า MACD Line
df['macd_signal'] = macd.macd_signal()  # ค่า Signal Line
df['macd_diff'] = macd.macd_diff()  # ค่า MACD different

print(df[['close', 'rsi', 'macd', 'macd_signal', 'macd_diff']].tail())

        close        rsi        macd  macd_signal   macd_diff
195  101382.0  51.003399  662.847001  1245.742221 -582.895220
196   97752.0  44.427636  304.983218  1057.590421 -752.607202
197   96569.6  42.505286  -73.192488   831.433839 -904.626327
198   96531.6  42.441727 -371.681233   590.810824 -962.492058
199   97397.4  44.478819 -532.237586   366.201142 -898.438729


In [56]:
# Drop missing value from the dataset
df.dropna(inplace=True)
print(df)

     timestamp      open      high      low     close        volume  \
33  2024-08-25   64158.3   65078.9  63782.0   64225.0  11005.461325   
34  2024-08-26   64225.1   64488.0  62773.4   62805.6  18705.509588   
35  2024-08-27   62805.5   63191.5  57910.0   59384.0  36681.830167   
36  2024-08-28   59384.0   60216.5  57800.0   59015.8  38464.206147   
37  2024-08-29   59015.8   61170.5  58500.8   59330.0  29528.106172   
..         ...       ...       ...      ...       ...           ...   
195 2025-02-03   97614.2  102511.0  91000.0  101382.0  74190.353191   
196 2025-02-04  101382.0  101776.5  96058.3   97752.0  40724.480363   
197 2025-02-05   97752.1   99177.5  96052.6   96569.6  22111.746163   
198 2025-02-06   96560.2   99137.9  95660.6   96531.6  18820.033799   
199 2025-02-07   96530.3   97846.9  96391.4   97397.4   5708.909622   

           rsi        macd  macd_signal   macd_diff  
33   58.816709   99.829828  -740.726861  840.556689  
34   54.044673  200.386414  -552.504206

In [57]:
#สร้าง Target Variable ถ้าราคาปิดของอีกวันสูงขึ้นจะเป็นการ บ่งบอกว่า Good Target
df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
print(df[['close', 'target']].tail())

        close  target
195  101382.0       0
196   97752.0       0
197   96569.6       0
198   96531.6       1
199   97397.4       0


In [58]:
# สร้าง feature เพิ่มเติม
# การเปลี่ยนแปลงของราคา
df['price_change'] = df['close'] - df['open']

### mention หากเกิดเวลาเปลี่ยนแปลง ต้องมาแก้ไข Function นี้

In [59]:
# splitting data สำหรับ test แบบ timeframe series
# แปลง timestamp เป็น datetime format (หากยังไม่ได้แปลง)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# แยกข้อมูล Train (เดือน 10, 11, 12 ปี 2024)
train_data = df[(df['timestamp'] >= '2024-10-01') & (df['timestamp'] <= '2024-12-31')]

# แยกข้อมูล Test (เดือน 1 ปี 2025)
test_data = df[(df['timestamp'] >= '2025-01-01') & (df['timestamp'] <= '2025-01-31')]

# ตรวจสอบข้อมูล
print("Train Data:")
print(train_data)
print("\nTest Data:")
print(test_data)

Train Data:
     timestamp     open     high      low    close        volume        rsi  \
70  2024-10-01  63285.8  64092.0  60080.2  60767.5  41315.097587  45.708757   
71  2024-10-02  60766.1  62355.0  59934.0  60616.0  26278.612640  45.232467   
72  2024-10-03  60610.2  61451.9  59815.1  60719.5  20929.881355  45.649136   
73  2024-10-04  60719.5  62461.0  60430.1  62057.4  20897.699948  50.854157   
74  2024-10-05  62057.4  62338.4  61655.2  62030.2   5697.061891  50.747752   
..         ...      ...      ...      ...      ...           ...        ...   
157 2024-12-27  95649.6  97415.7  93300.3  94140.1  33350.470651  43.809035   
158 2024-12-28  94140.0  95549.9  93964.4  95125.8   9792.543625  45.959150   
159 2024-12-29  95126.7  95172.7  92803.6  93519.2  12730.666810  43.066583   
160 2024-12-30  93519.2  94940.0  91271.5  92640.0  41588.112330  41.526304   
161 2024-12-31  92640.1  96200.0  91894.9  93422.5  27220.568128  43.464344   

            macd  macd_signal    macd_d

In [60]:
# ทำ Standardization (Z-Score Scaling) data ก่อน fitting model เพระาหน่วยแตกต่างกัน
# ปรับข้อมูลให้อยู่ในรูปค่าเฉลี่ย 0 และส่วนเบี่ยงเบนมาตรฐาน 1
from sklearn.preprocessing import StandardScaler

# เลือก Features ที่ต้องการ Normalize
features_to_scale = ['rsi', 'macd_diff', 'price_change', 'volume', 'close']
target_variable = 'target'

# สร้าง Scaler
scaler = StandardScaler()

# Fit และ Transform ข้อมูล
train_data.loc[:, features_to_scale] = scaler.fit_transform(train_data[features_to_scale])
test_data.loc[:, features_to_scale] = scaler.transform(test_data[features_to_scale])

In [61]:
train_data = train_data.loc[:, features_to_scale + [target_variable]]
print(train_data)

          rsi  macd_diff  price_change    volume     close  target
70  -1.459648  -0.065423     -1.302241  0.375501 -1.500731       0
71  -1.502724  -0.367827     -0.218507 -0.372464 -1.510761       1
72  -1.465040  -0.540965     -0.099801 -0.638527 -1.503909       1
73  -0.994295  -0.515149      0.462430 -0.640128 -1.415332       0
74  -1.003918  -0.483532     -0.162266 -1.396258 -1.417133       1
..        ...        ...           ...       ...       ...     ...
157 -1.631460  -1.411885     -0.840595 -0.020686  0.708740       1
158 -1.437002  -1.347880      0.301302 -1.192535  0.773999       0
159 -1.698607  -1.386859     -0.885441 -1.046383  0.667632       0
160 -1.837911  -1.421703     -0.552157  0.389081  0.609424       1
161 -1.662634  -1.304058      0.208223 -0.325608  0.661230       1

[92 rows x 6 columns]


In [62]:
# ระบุ Feature และ Target
features_to_scale = ['rsi', 'macd_diff', 'price_change', 'volume', 'close']
target_variable = 'target'  # เป้าหมายที่ต้องการพยากรณ์

# แยก X (Features) และ y (Target)
x_train = train_data[features_to_scale]
y_train = train_data[target_variable]

x_test = test_data[features_to_scale]
y_test = test_data[target_variable]

ทำการ Fit model แบบ classificatiion tasks

Logistic Regression

Random Forest

XGBoost

NN - show highest performance ตอนนี้


In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd

# สร้างและฝึก Logistic Regression
model = LogisticRegression()
model.fit(x_train, y_train)  # ฝึกโมเดล

# ดึงค่า Coefficient
coefficients = model.coef_[0]  # เนื่องจากเป็น Binary Classification จึงมีเพียง 1 ชุด
features = x_train.columns  # ชื่อ Features หาก x_train เป็น DataFrame

# แสดงผล Coefficients พร้อมชื่อ Feature
coef_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})
coef_df = coef_df.sort_values(by='Coefficient', ascending=False)
print("Coefficients of Features:")
print(coef_df)

# ทำนายผล
y_pred = model.predict(x_test)

# ประเมินผล
print("Classification Report:")
print(classification_report(y_test, y_pred))


Coefficients of Features:
        Feature  Coefficient
0           rsi     0.261958
3        volume     0.221719
2  price_change    -0.022830
4         close    -0.230090
1     macd_diff    -0.269051
Classification Report:
              precision    recall  f1-score   support

           0       0.47      1.00      0.64        14
           1       1.00      0.06      0.11        17

    accuracy                           0.48        31
   macro avg       0.73      0.53      0.37        31
weighted avg       0.76      0.48      0.35        31



In [64]:
import statsmodels.api as sm
import pandas as pd

# สร้างโมเดล Logistic Regression ด้วย Statsmodels
x_train_sm = sm.add_constant(x_train)  # เพิ่มคอลัมน์ constant (intercept)
model_sm = sm.Logit(y_train, x_train_sm).fit()  # ฝึก Logistic Regression

# ดูสรุปผลของโมเดล
print(model_sm.summary())

# ดึงค่า Coefficients และ Confidence Intervals
coefficients = model_sm.params  # Coefficients
conf_int = model_sm.conf_int(alpha=0.05)  # Confidence Intervals 95%
conf_int.columns = ['Lower 95% CI', 'Upper 95% CI']  # เปลี่ยนชื่อคอลัมน์

# รวมข้อมูล Coefficients และ Confidence Intervals ใน DataFrame
result_df = pd.concat([coefficients, conf_int], axis=1)
result_df.columns = ['Coefficient', 'Lower 95% CI', 'Upper 95% CI']

# แสดงผล
print(result_df)

Optimization terminated successfully.
         Current function value: 0.680561
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                   92
Model:                          Logit   Df Residuals:                       86
Method:                           MLE   Df Model:                            5
Date:                Fri, 07 Feb 2025   Pseudo R-squ.:                 0.01277
Time:                        18:07:34   Log-Likelihood:                -62.612
converged:                       True   LL-Null:                       -63.421
Covariance Type:            nonrobust   LLR p-value:                    0.8989
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.1783      0.211      0.844      0.399      -0.236       0.592
rsi              0.4348

Winning of model is FNN model focusing on tuning this model

ปรับ Threshold ของโมเดล Default Threshold สำหรับโมเดล Classification คือ 0.5 (ผลลัพธ์ Sigmoid หรือ Probability > 0.5 จะถือว่าเป็น Class 1) focus การ  False Positives โดยการเพิ่ม Threshold

In [71]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential([
    Input(shape=(x_train.shape[1],)),  # กำหนด Input Shape
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # สำหรับ Binary Classification
])

# คอมไพล์และฝึกโมเดล
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Precision'])
class_weights = {0: 1, 1: 1}  # เพิ่มน้ำหนักให้ Class 1 มากกว่า
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test), class_weight=class_weights)

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - Precision: 0.5456 - loss: 0.7105 - val_Precision: 0.4737 - val_loss: 0.7007
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - Precision: 0.5192 - loss: 0.7111 - val_Precision: 0.5000 - val_loss: 0.6991
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - Precision: 0.4877 - loss: 0.7113 - val_Precision: 0.3846 - val_loss: 0.6988
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - Precision: 0.5509 - loss: 0.7026 - val_Precision: 0.4286 - val_loss: 0.6975
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - Precision: 0.5326 - loss: 0.6957 - val_Precision: 0.5000 - val_loss: 0.6964
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - Precision: 0.5807 - loss: 0.6844 - val_Precision: 0.5000 - val_loss: 0.6954
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - Precision: 0.6237 - loss: 0.6782 - val_Precision: 0.5333 - val_loss: 0.6939
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - Precision: 0.5709 - loss: 0.6869 - val_Precision: 0.5833 - v

In [72]:
from sklearn.metrics import precision_score
import numpy as np

n_iterations = 100
precision_0_list = []
precision_1_list = []

for i in range(n_iterations):
    y_pred = (model.predict(x_test) > 0.5).astype(int)  # Threshold = 0.5

    # Calculate Precision with zero_division=0
    precision_0 = precision_score(y_test, y_pred, pos_label=0, zero_division=0)
    precision_1 = precision_score(y_test, y_pred, pos_label=1, zero_division=0)

    # Append to lists
    precision_0_list.append(precision_0)
    precision_1_list.append(precision_1)

# Calculate Mean and Standard Deviation
print(f"Precision for class 0: Mean = {np.mean(precision_0_list):.4f}")
print(f"Precision for class 1: Mean = {np.mean(precision_1_list):.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━

In [75]:
# เลือก 7 แถวล่าสุดจาก test_data
last_7_days = test_data.iloc[-7:][['rsi', 'macd_diff', 'price_change', 'volume', 'close']]
input_data = last_7_days.iloc[-1:].values 


# พยากรณ์ด้วยโมเดล
prediction = model.predict(input_data)

# แสดงผลการพยากรณ์
print("Prediction:", prediction)

# Threshold = 0.5 (ค่ากลาง)
threshold = 0.5
label = (prediction >= threshold).astype(int)
print("Label:", label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Prediction: [[0.46032634]]
Label: [[0]]
